In [7]:
#****************************************
# This juptyter notebook was created for the project, Deep Learning Grid Search for
# Diabetes Onset Prediction
# from the Udemy course, Aplied Machine Learning in Healthcare, 
#     https://www.udemy.com/course/applied-machine-learning-for-healthcare
#In this project we complete the following activities:
# 1. FILL THIS IN
#
#*****************************************************************
import sys
import pandas #to handle data sets
import numpy #for computation and arrays
import sklearn #for machine learning packages
import keras

print('Python: {}'.format(sys.version))
print('Pandas: {}'.format(pandas.__version__))
print('Numpy: {}'.format(numpy.__version__))
print('Sklearn: {}'.format(sklearn.__version__))
print('Keras: {}'.format(keras.__version__))



Python: 3.7.4 (default, Aug  9 2019, 18:34:13) [MSC v.1915 64 bit (AMD64)]
Pandas: 0.25.1
Numpy: 1.16.5
Sklearn: 0.21.3
Keras: 2.2.4


In [ ]:
# we'll be using these library's a lot, so lets make it easier to call them
import pandas as pd
import numpy as np

#import the uci pima indians diabetes dataset
url = "https://archive.ics.uci.edu/ml/machine-learning-databases/pima-indians-diabetes/pima-indians-diabetes.data"